In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import scipy.stats as sts
from datetime import datetime

In [ ]:
import plotly.express as px

def plotly_linechart(df, x_col, y_col, xtitle, ytitle, title, subtitle, color=None ):
    fig = px.line(df, x=x_col, y=y_col,color=color,
                  title=f'{title}<br><sup>{subtitle}</sup>',
                  labels={x_col: xtitle,y_col: ytitle})
    fig.show()

def plotly_barchart(df, x_col, y_col, xtitle, ytitle,  title, subtitle, color=None):
    fig = px.bar(df, x=x_col, y=y_col, color=color,
                 title=f'{title}<br><sup>{subtitle}</sup>',
                 labels={x_col: xtitle,y_col: ytitle})
    fig.show()

def plotly_piechart(df, val, names, title, subtitle, color=None):
    fig = px.pie(df, values=val, names=names, color=color, title=f'{title}<br><sup>{subtitle}</sup>')
    fig.show()

def plotly_histogram(df, values, title, subtitle, nbins=100, color=None):
    fig = px.histogram(df, x=values, nbins= nbins, color=color, title=f'{title}<br><sup>{subtitle}</sup>', text_auto=True)
    fig.show()

# Drivers

In [ ]:
drivers = pd.read_csv('/content/drivers.csv').replace('\\N', np.nan)
drivers['full_name'] = drivers['forename'] + ' ' + drivers['surname']
drivers['dob'] = drivers['dob'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
drivers_nationality = drivers.groupby('nationality').agg(count=pd.NamedAgg('nationality', 'size'),
                                                         names=pd.NamedAgg('full_name', pd.Series.unique)).sort_values('count', ascending=False).reset_index()

drivers_nationality['proportion'] = drivers_nationality['count'] / drivers.shape[0]
drivers_nationality['cumulative_proportion'] = drivers_nationality['proportion'].cumsum()
drivers_nationality = drivers_nationality.reset_index(drop=False, names='rank')
drivers_nationality['rank'] += 1

top10_nationalities = drivers_nationality[['nationality', 'count', 'cumulative_proportion']].iloc[:10]
top10_nationalities = pd.concat([top10_nationalities, pd.DataFrame([{'nationality': 'Other',
                                                                    'count': drivers.shape[0] - top10_nationalities['count'].sum(),
                                                                    'cumulative_proportion': 1 - top10_nationalities.iloc[9]['cumulative_proportion']}])])


plotly_piechart(top10_nationalities, val='count', names='nationality', title=f"Top 10 Driver's Nationalities (fraction: {round(drivers_nationality.iloc[9]['cumulative_proportion'], 3) * 100}%)", subtitle="Aggregated from 'drivers.csv'")
plotly_linechart(drivers_nationality, x_col='rank', y_col='cumulative_proportion',  xtitle='Top K Nationalities', ytitle='Proportion', title="Cumulative Fraction - Top K Driver's Nationalities in Formula 1", subtitle="Aggregated from 'drivers.csv'")

# Constructors

In [ ]:
constructors = pd.read_csv('/content/constructors.csv').replace('\\N', np.nan)
constructors

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri


# Circuits

In [ ]:
circuits = pd.read_csv('/content/circuits.csv').rename(columns={'name': 'circuit_name', 'url': 'circuit_url'}).replace('\\N', np.nan)

circuits

,circuitId,circuitRef,circuit_name,location,country,lat,lng,alt,circuit_url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,NaN,http://en.wikipedia.org/wiki/Losail_Internatio...


# Races

In [ ]:
from datetime import datetime

races = pd.read_csv('/content/races.csv').replace('\\N', np.nan).rename(columns={'name': 'race_name', 'url': 'race_url'})

date_cols = [x for x in races.columns if x.find('date') != -1]
time_cols = [x.replace('date', 'time') for x in date_cols]
date_time_cols = [(x,y) for x,y in zip(date_cols, time_cols)]

for (date,time) in date_time_cols:
    mod_time = time
    mod_date = date
    if time[0] != 't':
        mod_time = time.split('_')[-1]
    if date[0] == 'd':
        mod_date = 'race_' + date

    races[f'{mod_date}{mod_time}'] = races[date] + " " + races[time]
    races[f'{mod_date}{mod_time}'] = races[f'{mod_date}{mod_time}'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S') if pd.notna(x) else np.nan)

races = races.drop(columns=date_cols + time_cols)

races

,raceId,year,round,circuitId,race_name,race_url,race_datetime,fp1_datetime,fp2_datetime,fp3_datetime,quali_datetime,sprint_datetime
0,1,2009,1,1,Australian Grand Prix,http://en.wikipedia.org/wiki/2009_Australian_G...,2009-03-29 06:00:00,NaT,NaT,NaT,NaT,NaT
1,2,2009,2,2,Malaysian Grand Prix,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,2009-04-05 09:00:00,NaT,NaT,NaT,NaT,NaT
2,3,2009,3,17,Chinese Grand Prix,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,2009-04-19 07:00:00,NaT,NaT,NaT,NaT,NaT
3,4,2009,4,3,Bahrain Grand Prix,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,2009-04-26 12:00:00,NaT,NaT,NaT,NaT,NaT
4,5,2009,5,4,Spanish Grand Prix,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,2009-05-10 12:00:00,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
1096,1116,2023,18,69,United States Grand Prix,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-22 19:00:00,2023-10-20 17:30:00,2023-10-21 18:00:00,NaT,2023-10-20 21:00:00,2023-10-21 22:00:00
1097,1117,2023,19,32,Mexico City Grand Prix,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-29 20:00:00,2023-10-27 18:30:00,2023-10-27 22:00:00,2023-10-28 17:30:00,2023-10-28 21:00:00,NaT
1098,1118,2023,20,18,São Paulo Grand Prix,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-05 17:00:00,2023-11-03 14:30:00,2023-11-04 14:30:00,NaT,2023-11-03 18:00:00,2023-11-04 18:30:00
1099,1119,2023,21,80,Las Vegas Grand Prix,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-19 06:00:00,2023-11-17 04:30:00,2023-11-17 08:00:00,2023-11-18 04:30:00,2023-11-18 08:00:00,NaT


In [ ]:
results = pd.read_csv('/content/results.csv').replace('\\N', np.nan)

results = results.merge(races[['raceId','race_name', 'race_datetime']], how='inner', on='raceId')
# results = results.merge(circuits[['circuitId','circuit_name', 'location', 'country', 'alt']], how='inner', on='circuitId')

results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,race_name,race_datetime
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,Australian Grand Prix,2008-03-16 04:30:00
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,Australian Grand Prix,2008-03-16 04:30:00
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,Australian Grand Prix,2008-03-16 04:30:00
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,Australian Grand Prix,2008-03-16 04:30:00
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,Australian Grand Prix,2008-03-16 04:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26075,26081,1110,817,213,3,19,16,16,16,0.0,44,+1:43.071,5053521,25,15,1:50.994,227.169,1,Belgian Grand Prix,2023-07-30 13:00:00
26076,26082,1110,858,3,2,18,17,17,17,0.0,44,+1:44.476,5054926,37,9,1:50.486,228.213,1,Belgian Grand Prix,2023-07-30 13:00:00
26077,26083,1110,807,210,27,0,18,18,18,0.0,44,+1:50.450,5060900,26,4,1:49.907,229.415,1,Belgian Grand Prix,2023-07-30 13:00:00
26078,26084,1110,832,6,55,4,NaN,R,19,0.0,23,NaN,NaN,9,19,1:53.138,222.864,130,Belgian Grand Prix,2023-07-30 13:00:00


# Lap Times

In [ ]:
laps = pd.read_csv('/content/lap_times.csv').replace('\\N', np.nan).rename(columns={'time': 'laptime', 'milliseconds': 'laptime_ms'})
laps = laps.merge(races[['raceId', 'circuitId', 'year', 'round', 'race_name' ,'race_datetime']], how='inner', on='raceId')
laps = laps.merge(drivers[['driverId', 'code', 'full_name', 'dob','nationality']], how='inner', on='driverId')
laps = laps.merge(circuits[['circuitId', 'circuit_name', 'location', 'country', 'lat', 'lng', 'alt']], how='inner', on='circuitId')

laps.year.sort_values().unique()


array([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022, 2023])

In [ ]:
sample = laps[laps.raceId == 1035].copy()

sample['laptime_s'] = sample['laptime_ms'] / 1000

# race_sample_laptimes = sample.pivot(index='full_name', columns='lap', values='laptime_ms')

# race_sample_laptimes['finished'] =
# sns.heatmap(race_sample_laptimes)



In [ ]:
plotly_linechart(sample, "lap", "position", xtitle="Lap", ytitle="Position", color="full_name", title="Driver's Race Evolution (Position-wise)", subtitle=f"{sample.location.iloc[0]} - {sample.race_name.iloc[0]} {sample.year.iloc[0]}", )

In [ ]:
driver_laptimes = sample[(sample.full_name.isin(['Max Verstappen', 'Lewis Hamilton', 'Charles Leclerc', 'Lando Norris'])) & (sample.laptime_s < 105)].copy()

plotly_linechart(driver_laptimes, x_col="lap", y_col='laptime_s', xtitle='Lap', ytitle='Laptime (s)', color="full_name", title="Driver's Laptime Comparison", subtitle=f"{sample.location.iloc[0]} - {sample.race_name.iloc[0]} {sample.year.iloc[0]}")

In [ ]:
fig = px.violin(sample, y="laptime_s", color='full_name', box=False, # draw box plot inside the violin
                points='all', # can be 'outliers', or False
               )

fig.show()

In [ ]:
pattern = r'^\d{1,2}:\d{2}\.\d{3}$'

# Find entries that do not match the correct lap time format
invalid_laptimes = laps[~laps['laptime'].str.match(pattern)]

# Constructors

In [ ]:
constructors = pd.read_csv('/content/constructors.csv').replace('\\N', np.nan).rename(columns={'name': 'constructorName', 'nationality': 'constructorNationality', 'url': 'constructorURL'})

constructors

,constructorId,constructorRef,constructorName,constructorNationality,constructorURL
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri


# Qualifying

In [ ]:
def laptime2seconds(laptime):
    if isinstance(laptime, str):
        (mins, sec) = laptime.split(':')[-2:]
        (sec, ms) = sec.split('.')
        return 60*float(mins) + float(sec) + float(f'0.{ms}')
    return laptime

In [ ]:
qualifying = pd.read_csv('/content/qualifying.csv').replace('\\N', np.nan)
qualifying = qualifying.merge(races[['raceId', 'circuitId', 'year', 'round', 'race_name' ,'race_datetime']], how='inner', on='raceId')
qualifying = qualifying.merge(drivers[['driverId', 'code', 'full_name', 'dob','nationality']], how='inner', on='driverId')
qualifying = qualifying.merge(circuits[['circuitId', 'circuit_name', 'location', 'country', 'lat', 'lng', 'alt']], how='inner', on='circuitId')
qualifying = qualifying.merge(constructors[['constructorId', 'constructorName']], how='inner', on='constructorId')

for col in ['q1','q2', 'q3']:
    qualifying[f'secLaptime{col.capitalize()}'] = qualifying[col].apply(laptime2seconds)
    # tmp = qualifying.groupby(['raceId']).agg(bestLap=pd.NamedAgg(f'{col}Laptime', 'min'))

session_aggregation = qualifying.groupby(['raceId']).agg(bestLaptimeQ1=pd.NamedAgg('secLaptimeQ1', 'min'), bestLaptimeQ2=pd.NamedAgg('secLaptimeQ2', 'min'), bestLaptimeQ3=pd.NamedAgg('secLaptimeQ3', 'min'))

qualifying = qualifying.merge(session_aggregation, how='inner', on='raceId')

for col in ['q1','q2', 'q3']:
    qualifying[f'bestTimeDelta{col.capitalize()}'] = (qualifying[f'secLaptime{col.capitalize()}'] - qualifying[f'bestLaptime{col.capitalize()}'])

qualifying


,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,circuitId,...,constructorName,secLaptimeQ1,secLaptimeQ2,secLaptimeQ3,bestLaptimeQ1,bestLaptimeQ2,bestLaptimeQ3,bestTimeDeltaQ1,bestTimeDeltaQ2,bestTimeDeltaQ3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,1,...,McLaren,86.572,85.187,86.714,85.664,85.187,86.714,0.908,0.000,0.000
1,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,1,...,McLaren,85.664,85.452,87.079,85.664,85.187,86.714,0.000,0.265,0.365
2,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,1,...,BMW Sauber,86.103,85.315,86.869,85.664,85.187,86.714,0.439,0.128,0.155
3,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,1,...,BMW Sauber,85.960,85.518,87.236,85.664,85.187,86.714,0.296,0.331,0.522
4,12,18,4,4,5,12,1:26.907,1:26.188,NaN,1,...,Renault,86.907,86.188,NaN,85.664,85.187,86.714,1.243,1.001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9810,2932,270,101,31,31,25,1:27.201,NaN,NaN,26,...,Simtek,87.201,NaN,NaN,82.762,NaN,NaN,4.439,NaN,NaN
9811,2928,270,70,32,12,21,1:25.557,NaN,NaN,26,...,Team Lotus,85.557,NaN,NaN,82.762,NaN,NaN,2.795,NaN,NaN
9812,2929,270,103,32,11,22,1:25.595,NaN,NaN,26,...,Team Lotus,85.595,NaN,NaN,82.762,NaN,NaN,2.833,NaN,NaN
9813,2930,270,100,33,20,23,1:26.272,NaN,NaN,26,...,Larrousse,86.272,NaN,NaN,82.762,NaN,NaN,3.510,NaN,NaN


In [ ]:
drivers_qualifying = qualifying.groupby(['full_name', 'year']).agg(appeareancesQ1=pd.NamedAgg('q1', pd.Series.dropna),
                                                                                      appeareancesQ2=pd.NamedAgg('q2', pd.Series.dropna),
                                                                                      appeareancesQ3=pd.NamedAgg('q3', pd.Series.dropna),
                                                                                      avgSessionDeltaQ1=pd.NamedAgg('bestTimeDeltaQ1', 'mean'),
                                                                                      avgSessionDeltaQ2=pd.NamedAgg('bestTimeDeltaQ2', 'mean'),
                                                                                      avgSessionDeltaQ3=pd.NamedAgg('bestTimeDeltaQ3', 'mean'),
                                                                                      avgResultQualifying=pd.NamedAgg('position', 'mean'),
                                                                                      bestResultQualifying=pd.NamedAgg('position', 'min'),
                                                                                      worstResultQualifying=pd.NamedAgg('position', 'max'),
                                                                                      ).reset_index()

for col in ['q1','q2', 'q3']:
    drivers_qualifying[f'appeareances{col.capitalize()}'] = drivers_qualifying[f'appeareances{col.capitalize()}'].apply(lambda x: len(x))

In [ ]:
sample_drivers = ['Sebastian Vettel', 'Lewis Hamilton', 'Fernando Alonso', 'Kimi Räikkönen']

In [ ]:
plotly_linechart(drivers_qualifying[drivers_qualifying.full_name.isin(sample_drivers)],
                 x_col='year',
                 y_col='avgResultQualifying',
                 xtitle='Year',
                 ytitle='AVG Qualifying Position',
                 color='full_name',
                 title="Driver's Seasonal Average Qualifying Position",
                 subtitle=f'Drivers: {sample_drivers}')

# Descriptive Plots

In [ ]:
results = pd.read_csv('/content/results.csv').replace('\\N', np.nan)

results = results.merge(races[['raceId','race_name', 'race_datetime', 'year', 'round', 'circuitId']], how='inner', on='raceId')
results = results.merge(constructors[['constructorId','constructorName', 'constructorNationality']], how='inner', on='constructorId')
results = results.merge(drivers[['driverId','full_name', 'nationality', 'code', 'dob']], how='inner', on='driverId')

## Seasonal Statistics

In [ ]:
seasonal_statistics = results.groupby('year').agg(races=pd.NamedAgg('raceId', pd.Series.nunique),
                                                  drivers=pd.NamedAgg('driverId', pd.Series.nunique),
                                                  teams=pd.NamedAgg('constructorId', pd.Series.nunique),
                                                  total_points=pd.NamedAgg('points', 'sum'),
                                                  dnfs=pd.NamedAgg('position', lambda x: x.isna().sum()),
                                                  min_datetime=pd.NamedAgg('race_datetime', 'min'),
                                                  max_datetime=pd.NamedAgg('race_datetime', 'max'),
                                                  nationalities=pd.NamedAgg('nationality', pd.Series.nunique),
                                                  )

seasonal_statistics['drivers/team'] = seasonal_statistics['drivers'] / seasonal_statistics['teams']
seasonal_statistics['points/race'] = seasonal_statistics['total_points'] / seasonal_statistics['races']
seasonal_statistics['seasonal_length_days'] = (seasonal_statistics['max_datetime'] - seasonal_statistics['min_datetime']).dt.days
seasonal_statistics['mean_days_between_races'] = seasonal_statistics['seasonal_length_days'] / seasonal_statistics['races']


seasonal_statistics = seasonal_statistics.reset_index()

### Races per Season

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='races',
                 xtitle='Year',
                 ytitle='Races',
                 title='Formula 1 Races per Season (1950-2023)',
                 subtitle='Remark: Incomplete data for 2023. Should be 22 races')

### Season Length in Days

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='seasonal_length_days',
                 xtitle='Year',
                 ytitle='Seasonal Length (Days)',
                 title='Formula 1 Seasonal Lengths (1950-2023)',
                 subtitle='Seasonal length measured in days from first to last race. Note: the dataset is missing data information for all races before 2005.')

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='seasonal_length_days',
                 xtitle='Year',
                 ytitle='Seasonal Length (Days)',
                 title='Formula 1 Seasonal Lengths (1950-2023)',
                 subtitle='Seasonal length measured in days from first to last race. Note: the dataset is missing data information for all races before 2005.')

### Teams per Season

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='mean_days_between_races',
                 xtitle='Year',
                 ytitle='Days',
                 title='Formula 1 AVG Days Between Races (1950-2023)',
                 subtitle='Since 2005, there have been at least one race every second week during the season. Due to covid-19, the season was shortened and delayed to June, thus resulting in fewer days between races.'
                 )

### Drivers per Season

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='drivers',
                 xtitle='Year',
                 ytitle='Drivers',
                 title='Formula 1 Drivers per Season (1950-2023)',
                 subtitle='The number of drivers per season have steadily decreased'
                 )

### Drivers / Team

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='drivers/team',
                 xtitle='Year',
                 ytitle='Drivers/Team',
                 title='Formula 1 Drivers/Team per Season (1950-2023)',
                 subtitle='There have been 2-3 drivers per team on average since 1974'
                 )

### Points per Season

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='total_points',
                 xtitle='Year',
                 ytitle='Total Points',
                 title='Formula 1 Total Points per Season (1950-2023)',
                 subtitle='Longer race calendars and a new point scoring system in 2010 have caused an increase in total points per season. The drop in 2020 was caused by COVID-19, while the drop in 2023 is caused by incomplete data.'
                 )

### DNFs per Season

In [ ]:
plotly_linechart(seasonal_statistics,
                 x_col='year',
                 y_col='dnfs',
                 xtitle='Year',
                 ytitle='DNFs',
                 title='Formula 1 DNFs per Season (1950-2023)',
                 subtitle='In spite of longer race calendars in recent years, the number of DNFs have decreased, perhaps suggesting improved reliability.'
                 )

## Distributional Statistics

### Dataset Driver Nationalities

In [ ]:
drivers_nationality = drivers.groupby('nationality').agg(count=pd.NamedAgg('nationality', 'size'),
                                                         names=pd.NamedAgg('full_name', pd.Series.unique)).sort_values('count', ascending=False).reset_index()

drivers_nationality['proportion'] = drivers_nationality['count'] / drivers.shape[0]
drivers_nationality['cumulative_proportion'] = drivers_nationality['proportion'].cumsum()
drivers_nationality = drivers_nationality.reset_index(drop=False, names='rank')
drivers_nationality['rank'] += 1

top10_nationalities = drivers_nationality[['nationality', 'count', 'cumulative_proportion']].iloc[:10]
top10_nationalities = pd.concat([top10_nationalities, pd.DataFrame([{'nationality': 'Other',
                                                                    'count': drivers.shape[0] - top10_nationalities['count'].sum(),
                                                                    'cumulative_proportion': 1 - top10_nationalities.iloc[9]['cumulative_proportion']}])])


plotly_piechart(top10_nationalities,
                val='count',
                names='nationality',
                title=f"Driver's Nationalities",
                subtitle=f"Top 10 Nationalities accounts for {round(drivers_nationality.iloc[9]['cumulative_proportion'], 3) * 100}% of the drivers that have raced in Formula 1 since 1950")

plotly_linechart(drivers_nationality, x_col='rank', y_col='cumulative_proportion',  xtitle='Top K Nationalities', ytitle='Proportion', title="Cumulative Fraction - Top K Driver's Nationalities in Formula 1", subtitle="Aggregated from 'drivers.csv'")

In [ ]:
driver_statistics = results.groupby('driverId').agg(dob=pd.NamedAgg('dob', 'min'),
                                                    first_race=pd.NamedAgg('race_datetime', 'min'),
                                                    last_race=pd.NamedAgg('race_datetime', 'max'),
                                                    seasons=pd.NamedAgg('year', pd.Series.nunique),
                                                    races=pd.NamedAgg('raceId', pd.Series.nunique),
                                                    total_points=pd.NamedAgg('points', 'sum'),
                                                    dnfs=pd.NamedAgg('position', lambda x: x.isna().sum()),
                                                    )

driver_statistics = driver_statistics.reset_index()

driver_statistics[driver_statistics.seasons > 3].shape[0]

299

In [ ]:
relevant_frac = driver_statistics[driver_statistics.seasons > 3].shape[0] / driver_statistics.shape[0]

plotly_histogram(driver_statistics, values='seasons', title='Formula 1 Seasons per Driver', subtitle=f'Only {round(relevant_frac*100, 2)}% of the {driver_statistics.shape[0]} drivers that have raced in Formula 1 got to race for more than 3 seasons.', nbins=20)

In [ ]:
relevant_frac = driver_statistics[driver_statistics.races > 20].shape[0] / driver_statistics.shape[0]

plotly_histogram(driver_statistics, values='races', title='Formula 1 Races per Driver', subtitle=f'Only {round(relevant_frac*100, 2)}% of the {driver_statistics.shape[0]} drivers that have raced in Formula 1 got to race for more than 20 races (approximately 1 race calendar in recent years).', nbins=30)

In [ ]:
filtered_ds = driver_statistics.dropna().copy()
filtered_ds['debut_age'] = ((filtered_ds['first_race'] - filtered_ds['dob']) / pd.Timedelta(days=365.25)).astype(int)
filtered_ds['retiring_age'] = ((filtered_ds['last_race'] - filtered_ds['dob']) / pd.Timedelta(days=365.25)).astype(int)

relevant_frac = filtered_ds[(filtered_ds.debut_age > 20) & (filtered_ds.debut_age < 25)].shape[0] / filtered_ds.shape[0]
plotly_histogram(filtered_ds, values='debut_age', title='Formula 1 Debut Age', subtitle=f'{round(relevant_frac*100, 2)}% of the {filtered_ds.shape[0]} drivers we have debut age for, debuted at age between 20 and 25', nbins=20)

In [ ]:
relevant_frac = filtered_ds[filtered_ds.debut_age < 30].shape[0] / filtered_ds.shape[0]
plotly_histogram(filtered_ds, values='retiring_age', title='Formula 1 Debut Age', subtitle=f'{round(relevant_frac*100, 2)}% of the {filtered_ds.shape[0]} drivers we have retiring age for, completed their last race before 30', nbins=20)